# GROUP PROJECT

# Introduction

The data for this project comes from a research Minecraft server run by a UBC Computer Science group. This server specifically is mainly used to study how people behave in open-world games. We have access to 2 files, but in this report we are going to only use the file called **players.csv**. This file has one row per player and includes the main account and demographic details. The fields that have been stated describe each player at a very basic level and it gives us enough information to explore the differences between subscribers and non-subscribers.

## Dataset Description

Here are the fields we use to describe each player:

| Variable name | Type | Meaning |
|---------------|------|---------|
| `player_id` | character | Unique ID for each player |
| `subscribed_newsletter` | logical / integer | Whether the player subscribed to the newsletter |
| `played_hours` *(if present)* | numeric | Total hours the player has played |
| `experience` *(if present)* | character | Self-reported experience level |
| `gender` *(if present)* | character | Player gender |
| `Age` *(if present)* | numeric | Player age |
| *(other variables)* | — | Additional account-level information |

## Project Question

One of the goals of the research team is to figure out what kind of players are most likely to subscribe to the project’s email newsletter. Since subscribing is optional, it can act as a sign of interest or engagement. If certain patterns in player characteristics predict who subscribes, then the team can make better choices about recruitment and resource planning.

In this report, we focus on the question:  
**Which player characteristics help predict whether a player subscribes to the newsletter?**

## Approach

To study this question, we use only the variables available in **players.csv**. These include things like total playtime, experience level, age, and gender. We build a single table with one row per player and then explore how these characteristics relate to newsletter subscription. After the exploratory analysis, we fit models to see how well these features can predict whether someone subscribes.





In [ ]:
HELLO